In [ ]:
import h5py
import random
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt

from Functions.image_preprocessing import *

In [ ]:
PATH_3D_H5 = 'Images/tof_data_128_112_40_outcome.hdf5'

def decode_data(string):
    decoded_string = [n.decode("UTF-8", "ignore") for n in string]
    return(decoded_string)

with h5py.File(PATH_3D_H5, 'r') as h5:
    print('H5-file: ', list(h5.keys()))
    
    # Image matrices
    X = h5["X"][:]
    # Patient ID's
    pat = h5["pat"][:]
    # Path to images
    path = decode_data(h5["path"][:])
    # Patient labels (1=bad outcome, 0=good outcome)
    Y_outcome = h5["stroke"][:]
    
print(len(X), len(Y_outcome), len(pat), len(path))

#### Expand dimensions

In [ ]:
X = np.expand_dims(X, axis = 4)
X.shape

#### Values between 0 and 255

In [ ]:
X_norm = np.empty_like(X)

for i in range(X.shape[0]):
    X_norm[i] = normalize(X[i])  
    
print(X_norm.shape, X_norm.min(), X_norm.max())

#### Z-Standardization

In [ ]:
X_normz = np.array([z_normalize(img) for img in X_norm])
print(X_normz.shape, X_normz.min(), X_normz.max(), X_normz.mean(), X_normz.std())

In [ ]:
fig = plt.figure(figsize = (20, 40)) # total figure size (including all subplots)
fig_all = []
img = np.squeeze(X_normz[0], axis = 3)
for i in range(1, 39):
    image = img[:,:,i]
    fig_all.append(fig.add_subplot(8, 5, i))
    plt.imshow(image, cmap = 'gray')
plt.show()

#### Save in hdf5 file

In [ ]:
# save the preprocessed data
with h5py.File('Images/preprocessed_data_128_112_40_outcome.hdf5', "w") as h5:
    h5.create_dataset("X", data = X_normz)
    h5.create_dataset("Y_outcome", data = Y_outcome)
    h5.create_dataset("pat", data = pat)